# Movie Recommendation System

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data Collection and Data Preprocessing

In [6]:
# loading data from csv file to pandas dataframes
import csv
df=pd.read_csv('/content/movies.csv')
df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800.0,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500.0,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466.0,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106.0,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124.0,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3798,3798,4000000,Comedy Drama,NaN,60422,NaN,en,Another Happy Day,A wedding at her parents' Annapolis estate hur...,3.238466,...,119.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"At this wedding, the F-word stands for Family",Another Happy Day,6.2,37.0,Ellen Barkin Ezra Miller Kate Bosworth Demi Mo...,"[{'name': 'Sam Levinson', 'gender': 0, 'depart...",Sam Levinson
3799,3799,4000000,Adventure Action Thriller Crime,NaN,81390,terror scotland kidnapping nudity wilderness,en,A Lonely Place to Die,A group of five mountaineers are hiking and cl...,8.307230,...,99.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"Out here, there's nowhere to hide.",A Lonely Place to Die,6.2,157.0,Melissa George Ed Speleers Eamonn Walker Alec ...,"[{'name': 'Steve Daly', 'gender': 0, 'departme...",Julian Gilbey
3800,3800,0,Comedy Fantasy Science Fiction,NaN,10981,loser bullying leere independent film best friend,en,Nothing,The film tells the story of two good friends w...,3.408955,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Nothing,5.8,38.0,David Hewlett Andrew Miller Gordon Pinsent Mar...,"[{'name': 'Michael Andrews', 'gender': 2, 'dep...",Vincenzo Natali
3801,3801,0,Drama,NaN,225235,alcohol adultery river camping province,ru,Географ глобус пропил,Victor Sluzhkin signs on as a teacher of geogr...,1.8

In [7]:
# Printing first five rows of dataframe
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800.0,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500.0,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466.0,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106.0,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124.0,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [8]:
# Printing the last five rows of the dataframe
df.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
3798,3798,4000000,Comedy Drama,NaN,60422,NaN,en,Another Happy Day,A wedding at her parents' Annapolis estate hur...,3.238466,...,119.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"At this wedding, the F-word stands for Family",Another Happy Day,6.2,37.0,Ellen Barkin Ezra Miller Kate Bosworth Demi Mo...,"[{'name': 'Sam Levinson', 'gender': 0, 'depart...",Sam Levinson
3799,3799,4000000,Adventure Action Thriller Crime,NaN,81390,terror scotland kidnapping nudity wilderness,en,A Lonely Place to Die,A group of five mountaineers are hiking and cl...,8.307230,...,99.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"Out here, there's nowhere to hide.",A Lonely Place to Die,6.2,157.0,Melissa George Ed Speleers Eamonn Walker Alec ...,"[{'name': 'Steve Daly', 'gender': 0, 'departme...",Julian Gilbey
3800,3800,0,Comedy Fantasy Science Fiction,NaN,10981,loser bullying leere independent film best friend,en,Nothing,The film tells the story of two good friends w...,3.408955,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Nothing,5.8,38.0,David Hewlett Andrew Miller Gordon Pinsent Mar...,"[{'name': 'Michael Andrews', 'gender': 2, 'dep...",Vincenzo Natali
3801,3801,0,Drama,NaN,225235,alcohol adultery river camping province,ru,Географ глобус пропил,Victor Sluzhkin signs on as a teacher of geogr...,1.876621,...,120.0,"[{""iso_639_1"": ""ru"", ""name"": ""P\u0443\u0441\u0...",Released,NaN,The Geographer Drank His Globe Away,7.3,23.0,Konstantin Khabenskiy Elena Lyadova Aleksandr ...,"[{'name': 'Valery Todorovsky', 'gender': 2, 'd...",Aleksandr Veledinskiy
3802,3802,0,Drama Comedy History Music,NaN,14902,usa c,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Displaying the number of rows and columns of dataframe
df.shape

(3803, 24)

In [10]:
df.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,3803.000000,3.803000e+03,3803.000000,3802.000000,3.802000e+03,3801.000000,3802.000000,3802.000000
mean,1901.000000,3.638530e+07,47787.278727,25.182017,1.019841e+08,109.975796,6.185087,829.418201
std,1097.975865,4.282385e+07,78766.363425,34.188292,1.773457e+08,20.986172,0.915591,1340.549089
min,0.000000,0.000000e+00,5.000000,0.000372,0.000000e+00,0.000000,0.000000,0.000000
25%,950.500000,8.000000e+06,8370.000000,7.379789,1.756222e+06,96.000000,5.700000,100.250000
50%,1901.000000,2.200000e+07,12277.000000,16.667347,3.763116e+07,106.000000,6.200000,335.000000
75%,2851.500000,5.000000e+07,45611.000000,32.572093,1.196099e+08,120.000000,6.800000,927.000000
max,3802.000000,3.800000e+08,447027.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3803 entries, 0 to 3802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 3803 non-null   int64  
 1   budget                3803 non-null   int64  
 2   genres                3803 non-null   object 
 3   homepage              1421 non-null   object 
 4   id                    3803 non-null   int64  
 5   keywords              3577 non-null   object 
 6   original_language     3802 non-null   object 
 7   original_title        3802 non-null   object 
 8   overview              3801 non-null   object 
 9   popularity            3802 non-null   float64
 10  production_companies  3802 non-null   object 
 11  production_countries  3802 non-null   object 
 12  release_date          3802 non-null   object 
 13  revenue               3802 non-null   float64
 14  runtime               3801 non-null   float64
 15  spoken_languages     

In [12]:
df.dtypes

index                     int64
budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                 float64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count              float64
cast                     object
crew                     object
director                 object
dtype: object

In [22]:
# selecting the relevant features for recommendation
selected_features=['genres','keywords','title','tagline','director']

In [23]:
# replacing the null valuess with null string

for feature in selected_features:
  df[feature] = df[feature].fillna('')

In [24]:
# combining all the 5 selected features

combined_features = df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director']

In [25]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
3798    Comedy Drama  At this wedding, the F-word stan...
3799    Adventure Action Thriller Crime terror scotlan...
3800    Comedy Fantasy Science Fiction loser bullying ...
3801    Drama alcohol adultery river camping province ...
3802                                                  NaN
Length: 3803, dtype: object


In [26]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [29]:
combined_features = [str(feature) for feature in combined_features]
feature_vectors = vectorizer.fit_transform(combined_features)

In [30]:
print(feature_vectors)

  (0, 1955)	0.17325808891276728
  (0, 6370)	0.11499267777948705
  (0, 10703)	0.1967992090221845
  (0, 8395)	0.16169837494577272
  (0, 7185)	0.2296627090227083
  (0, 12012)	0.15282238422201544
  (0, 13732)	0.19804791484117854
  (0, 11558)	0.20357519610117245
  (0, 10938)	0.21411180371382496
  (0, 14250)	0.20672870277406405
  (0, 14029)	0.23314874891759035
  (0, 10963)	0.1497279083891937
  (0, 9425)	0.26949828881299615
  (0, 9174)	0.09074110662959092
  (0, 14021)	0.12649979551885573
  (0, 12557)	0.07036243677389624
  (0, 4026)	0.2370458099624692
  (0, 11732)	0.21411180371382496
  (0, 2600)	0.2465642825643519
  (0, 13661)	0.12329309465913228
  (0, 11824)	0.3354616153055467
  (0, 4789)	0.1648094898420996
  (0, 2470)	0.2265092023498167
  (0, 2984)	0.2296627090227083
  (0, 4445)	0.10320179465216495
  :	:
  (3800, 4470)	0.148226209395761
  (3800, 2621)	0.06926190057331344
  (3800, 3154)	0.12008339630671715
  (3800, 518)	0.1831871764316114
  (3800, 4445)	0.10834965749983594
  (3800, 11170)	0.1

### To Recommend the movie we have to use Cosine Similarity

In [31]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [32]:
print(similarity)

[[1.         0.06982297 0.03673202 ... 0.03461542 0.         0.        ]
 [0.06982297 1.         0.03134024 ... 0.01074153 0.         0.        ]
 [0.03673202 0.03134024 1.         ... 0.         0.         0.        ]
 ...
 [0.03461542 0.01074153 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


### Getting the movie name from the user

In [33]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : iron man


In [35]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = df['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [36]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [37]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [38]:
# finding the index of the movie with title

index_of_the_movie = df[df.title == close_match]['index'].values[0]
print(index_of_the_movie)

68


In [40]:
df.title[68]

'Iron Man'

In [41]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.032386996765788253), (1, 0.054732027237567996), (2, 0.012777128569029076), (3, 0.0060451163753774), (4, 0.03149764973956762), (5, 0.012835767117063066), (6, 0.07716464487951907), (7, 0.2376294162085494), (8, 0.007227706911439065), (9, 0.07459839712631763), (10, 0.07494888501133909), (11, 0.010991078911025777), (12, 0.012695157952833838), (13, 0.011347271855336141), (14, 0.09533038683060381), (15, 0.006653402890088154), (16, 0.22550152370111418), (17, 0.012167656919373084), (18, 0.04045833891752438), (19, 0.07817305207522163), (20, 0.07645751825594041), (21, 0.01040962355340954), (22, 0.006362984013477211), (23, 0.0060592964848213), (24, 0.011532983908209428), (25, 0.0), (26, 0.21393300865856762), (27, 0.029517826678698404), (28, 0.06082668339027722), (29, 0.013029250992465019), (30, 0.07738782905891803), (31, 0.30832937136726607), (32, 0.02854315228545617), (33, 0.12748453799561188), (34, 0.0), (35, 0.03427504129001759), (36, 0.0307381217299286), (37, 0.007331122498232018), (38,

In [42]:
len(similarity_score)

3803

In [43]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(68, 1.0000000000000002), (79, 0.40379182660763396), (31, 0.30832937136726607), (7, 0.2376294162085494), (16, 0.22550152370111418), (26, 0.21393300865856762), (85, 0.20358489038657382), (182, 0.19247495505908488), (511, 0.16496430970486958), (3623, 0.16072854376936077), (64, 0.15092905117742403), (203, 0.14555534743794216), (174, 0.14381137978831626), (101, 0.14117238492589082), (46, 0.1386933717314303), (169, 0.133523187409471), (1740, 0.1335173071394344), (94, 0.1316566698117334), (126, 0.13085208846797108), (788, 0.13055560902896882), (131, 0.1291105778266926), (33, 0.12748453799561188), (783, 0.12058273330371219), (138, 0.11918968661154154), (2487, 0.11854557201846272), (2442, 0.1180151868442773), (661, 0.11596145845049866), (607, 0.1106915450662149), (353, 0.11047994349346846), (2651, 0.10922181792687238), (38, 0.10882680996851699), (1451, 0.10778898906995438), (1553, 0.10626123696441003), (122, 0.10458651475149601), (242, 0.10229717621472234), (618, 0.1012018987042629), (1210, 0

In [45]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . Made
11 . X-Men: Apocalypse
12 . X2
13 . The Incredible Hulk
14 . X-Men: First Class
15 . X-Men: Days of Future Past
16 . Captain America: The First Avenger
17 . Kick-Ass 2
18 . Guardians of the Galaxy
19 . Thor: The Dark World
20 . Deadpool
21 . G-Force
22 . X-Men: The Last Stand
23 . Mortdecai
24 . The Last Airbender
25 . Duets
26 . Southland Tales
27 . Zathura: A Space Adventure
28 . Sky Captain and the World of Tomorrow
29 . Tropic Thunder


### Now finally we will movie recommendation system

In [47]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = df[df.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : avengers
Movies suggested for you : 

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Captain America: The Winter Soldier
4 . Captain America: Civil War
5 . Iron Man 2
6 . Thor: The Dark World
7 . X-Men
8 . The Incredible Hulk
9 . X-Men: Apocalypse
10 . Ant-Man
11 . Thor
12 . X2
13 . X-Men: The Last Stand
14 . Deadpool
15 . X-Men: Days of Future Past
16 . Captain America: The First Avenger
17 . The Amazing Spider-Man 2
18 . Iron Man
19 . Iron Man 3
20 . Man of Steel
21 . Superman II
22 . X-Men: First Class
23 . The Spirit
24 . Batman v Superman: Dawn of Justice
25 . Guardians of the Galaxy
26 . Serenity
27 . Spawn
28 . Teenage Mutant Ninja Turtles: Out of the Shadows
29 . What's Your Number?
